# lab11-2_CNN MNIST- 99percent
lab11-1 에서 진행한 CNN에서 CONV 와 POOL layer 를 하나씩 더 준다면.
![lec11_25](../../img/lec11_25.png)

In [11]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch import nn, optim
import numpy as np
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.nn.init

In [12]:
#MNIST data
mnist_trainset = datasets.MNIST(root = './data',train =True,transform=transforms.ToTensor(),download=True)
mnist_testset = datasets.MNIST(root = './data', train = False, transform=transforms.ToTensor(),download=True)

In [13]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
keep_prob = 0.7

# Data Loader (Input Pipeline)
train_loader = DataLoader(dataset=mnist_trainset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = DataLoader(dataset=mnist_testset,
                                          batch_size=batch_size,
                                          shuffle=False)

## Conv layer 1

fillter : [1,32] -> 각각 한장씩 32개 filter<br>
kernel_size : 3\*3짜리
stride : 1 -> 한칸씩 이동<br>
padding : 1 -> 각각 1씩 padding<br>
relu : ReLU 적용<br>
pool : kernel size : 1장 당 2 하나씩, strides : 2 로 두칸씩<br>
![lec11_26](../../img/lec11_26.png)

## Conv layer 2

fillter : [32,64] -> 3\*3 짜리 흑백 image 로 각각 한장씩 64개 filter
<br>
여기서, 앞의 Input 으로부터 depth 가 32이므로, 이를 받는다.
<br>
stride : 1 -> 한칸씩 이동<br>
padding : 1<br>
relu : ReLU 적용<br>
pool : kernel size : 1장 당 2 하나씩, strides : 2 로 두칸씩<br>
![lec11_27](../../img/lec11_27.png)

## Fully Connected(FC, Dense) Layer
Input : [7\*7\*64] -> [1\* 3136] 의  vector data<br>
Output_classes : 10<br>
bias : True <br>
Init method : Xavier_initializer<br>
Cost : softmax<br>
optimizer : Adam

![lec11_28](../../img/lec11_28.png)

In [22]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # Layer #1
        self.layer1 = nn.Sequential(nn.Conv2d(1,32,kernel_size = 3, stride = 1, padding =1),
                                    nn.ReLU(),
                                   nn.MaxPool2d(kernel_size = 2, stride =2)
                                   )
        # Layer #2
        self.layer2 = nn.Sequential(nn.Conv2d(32,64,kernel_size = 3, stride = 1, padding =1),
                                    nn.ReLU(),
                                   nn.MaxPool2d(kernel_size = 2, stride =2)
                                   )        
        self.fc = nn.Linear(7*7*64,10,bias=True)
        nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        in_size = x.size(0)
        output = self.layer1(x)
        output = self.layer2(output)
        output = output.view(in_size,-1)
        output = self.fc(output)

        return F.log_softmax(output,dim=1)


model = Model()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [23]:
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(len(mnist_trainset) / batch_size)
    model.train()

    for idx, (batch_xs, batch_ys) in enumerate(train_loader):
        optimizer.zero_grad()
        y_pred = model(batch_xs)
        loss = criterion(y_pred, batch_ys)
        loss.backward()
        optimizer.step()
        avg_cost += loss / total_batch
    print('\nTrain Epoch: {}\tLoss: {:.6f}'.format(epoch, avg_cost))
    

#         if idx % 10 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, idx * len(data), len(train_loader.dataset),
#                 100. * idx / len(train_loader), loss.data[0]))
    # TEST
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 0	Loss: 0.245029
Test set: Average loss: 0.0007, Accuracy: 9772/10000 (97%)


Train Epoch: 1	Loss: 0.062521
Test set: Average loss: 0.0004, Accuracy: 9877/10000 (98%)


Train Epoch: 2	Loss: 0.045249
Test set: Average loss: 0.0004, Accuracy: 9878/10000 (98%)


Train Epoch: 3	Loss: 0.037146
Test set: Average loss: 0.0004, Accuracy: 9891/10000 (98%)


Train Epoch: 4	Loss: 0.029526
Test set: Average loss: 0.0003, Accuracy: 9901/10000 (99%)


Train Epoch: 5	Loss: 0.025352
Test set: Average loss: 0.0003, Accuracy: 9889/10000 (98%)


Train Epoch: 6	Loss: 0.021623
Test set: Average loss: 0.0003, Accuracy: 9907/10000 (99%)


Train Epoch: 7	Loss: 0.017671
Test set: Average loss: 0.0004, Accuracy: 9877/10000 (98%)


Train Epoch: 8	Loss: 0.015612
Test set: Average loss: 0.0004, Accuracy: 9873/10000 (98%)


Train Epoch: 9	Loss: 0.012256
Test set: Average loss: 0.0003, Accuracy: 9914/10000 (99%)


Train Epoch: 10	Loss: 0.010198
Test set: Average loss: 0.0004, Accuracy: 9884/10000 (98%)